# Maximum Accuracy Model Training Strategy
## Advanced Stacked Ensemble for Drug Category Classification

**Objective:** Achieve maximum accuracy using state-of-the-art ensemble methods

**Strategy:**
- Layer 1: Multiple diverse base models (XGBoost, LightGBM, CatBoost, RF, ExtraTrees, NN)
- Layer 2: Stacked meta-models
- Bayesian hyperparameter optimization
- 10-fold stratified cross-validation

## 1. Setup and Imports

In [1]:
# Core libraries
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn preprocessing and metrics
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_validate
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

# Base Models
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

# Boosting models
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier

# Stacking
from sklearn.ensemble import StackingClassifier, VotingClassifier

# Hyperparameter optimization
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
try:
    import optuna
    OPTUNA_AVAILABLE = True
except:
    OPTUNA_AVAILABLE = False
    print("Optuna not available. Using GridSearchCV/RandomizedSearchCV only.")

# Utilities
import joblib
from datetime import datetime
import os

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("All libraries imported successfully!")
print(f"Optuna available: {OPTUNA_AVAILABLE}")

All libraries imported successfully!
Optuna available: True


## 2. Load Engineered Data

In [2]:
# Load engineered datasets
train_df = pd.read_csv('data/data_minihackathon_train_engineered.csv')
test_df = pd.read_csv('data/data_minihackathon_test_engineered.csv')

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print(f"\nTarget distribution:")
print(train_df['drug_category'].value_counts())

# Separate features and target
X = train_df.drop(['drug_category', 'id'], axis=1, errors='ignore')
y = train_df['drug_category']
X_test = test_df.drop(['id'], axis=1, errors='ignore')
test_ids = test_df['id'] if 'id' in test_df.columns else np.arange(len(test_df))

# Encode target variable
le = LabelEncoder()
y_encoded = le.fit_transform(y)

print(f"\nFeatures shape: {X.shape}")
print(f"Test features shape: {X_test.shape}")
print(f"\nClass encoding: {dict(enumerate(le.classes_))}")

Train shape: (1500, 46)
Test shape: (377, 45)

Target distribution:
drug_category
Hallucinogens    691
Stimulants       567
Depressants      242
Name: count, dtype: int64

Features shape: (1500, 45)
Test features shape: (377, 45)

Class encoding: {0: 'Depressants', 1: 'Hallucinogens', 2: 'Stimulants'}


## 3. Cross-Validation Setup

In [3]:
# Stratified K-Fold for cross-validation
N_FOLDS = 10
RANDOM_STATE = 42

skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)

# Storage for model results
model_results = {}
oof_predictions = {}  # Out-of-fold predictions for stacking

print(f"Using {N_FOLDS}-fold stratified cross-validation")
print(f"Random state: {RANDOM_STATE}")

Using 10-fold stratified cross-validation
Random state: 42


## 4. Layer 1: Base Models
### 4.1 XGBoost with Bayesian Optimization

In [ ]:
%%time

print("\n" + "="*80)
print("Training XGBoost with Hyperparameter Tuning")
print("="*80)

if OPTUNA_AVAILABLE:
    # Bayesian optimization with Optuna
    def objective_xgb(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'max_depth': trial.suggest_int('max_depth', 3, 11),
            'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'gamma': trial.suggest_float('gamma', 0, 0.3),
            'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
            'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
            'random_state': RANDOM_STATE,
            'tree_method': 'gpu_hist',
            'device': 'cuda',
            'eval_metric': 'mlogloss'
        }
        
        model = xgb.XGBClassifier(**params)
        scores = cross_val_score(model, X, y_encoded, cv=skf, scoring='accuracy', n_jobs=-1)
        return scores.mean()
    
    study_xgb = optuna.create_study(direction='maximize', study_name='xgboost')
    study_xgb.optimize(objective_xgb, n_trials=100, show_progress_bar=True)
    
    print(f"\nBest XGBoost accuracy: {study_xgb.best_value:.4f}")
    print(f"Best parameters: {study_xgb.best_params}")
    
    # Train final model with best parameters
    xgb_model = xgb.XGBClassifier(**study_xgb.best_params)
    
else:
    # Randomized search as fallback
    param_dist = {
        'n_estimators': [100, 300, 500, 700, 1000],
        'max_depth': [3, 5, 7, 9, 11],
        'learning_rate': [0.001, 0.01, 0.05, 0.1],
        'min_child_weight': [1, 3, 5, 7],
        'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
        'gamma': [0, 0.1, 0.2, 0.3],
        'reg_alpha': [0, 0.01, 0.1, 1],
        'reg_lambda': [0, 0.01, 0.1, 1]
    }
    xgb_base = xgb.XGBClassifier(random_state=RANDOM_STATE, tree_method='gpu_hist', device='cuda', eval_metric='mlogloss')
    xgb_base = xgb.XGBClassifier(random_state=RANDOM_STATE, tree_method='hist', eval_metric='mlogloss')
    random_search = RandomizedSearchCV(xgb_base, param_dist, n_iter=100, cv=5, 
                                      scoring='accuracy', n_jobs=-1, random_state=RANDOM_STATE)
    random_search.fit(X, y_encoded)
    
    print(f"\nBest XGBoost accuracy: {random_search.best_score_:.4f}")
    print(f"Best parameters: {random_search.best_params_}")
    
    xgb_model = random_search.best_estimator_

# Evaluate with cross-validation
cv_results = cross_validate(xgb_model, X, y_encoded, cv=skf, 
                           scoring=['accuracy', 'f1_macro'], return_train_score=True)

model_results['XGBoost'] = {
    'cv_accuracy': cv_results['test_accuracy'].mean(),
    'cv_std': cv_results['test_accuracy'].std(),
    'f1_macro': cv_results['test_f1_macro'].mean(),
    'model': xgb_model
}

print(f"\nXGBoost CV Accuracy: {model_results['XGBoost']['cv_accuracy']:.4f} ± {model_results['XGBoost']['cv_std']:.4f}")
print(f"XGBoost F1-Macro: {model_results['XGBoost']['f1_macro']:.4f}")

[I 2025-11-16 12:58:15,045] A new study created in memory with name: xgboost



Training XGBoost with Hyperparameter Tuning


Best trial: 0. Best value: 0.707333:   1%|          | 1/100 [00:07<13:00,  7.88s/it]

[I 2025-11-16 12:58:22,922] Trial 0 finished with value: 0.7073333333333334 and parameters: {'n_estimators': 409, 'max_depth': 6, 'learning_rate': 0.009013223892741149, 'min_child_weight': 7, 'subsample': 0.7653409673712619, 'colsample_bytree': 0.9757172827484016, 'gamma': 0.26627155882963194, 'reg_alpha': 0.1039928170729526, 'reg_lambda': 0.7341218660122519}. Best is trial 0 with value: 0.7073333333333334.


Best trial: 0. Best value: 0.707333:   2%|▏         | 2/100 [00:13<10:43,  6.56s/it]

...

[I 2025-11-16 13:04:15,626] Trial 97 finished with value: 0.7126666666666666 and parameters: {'n_estimators': 685, 'max_depth': 4, 'learning_rate': 0.006072205250493584, 'min_child_weight': 2, 'subsample': 0.851008465303482, 'colsample_bytree': 0.750380241895864, 'gamma': 0.16310332256256216, 'reg_alpha': 0.7276563763688106, 'reg_lambda': 0.9193081197022341}. Best is trial 51 with value: 0.7173333333333333.


Best trial: 51. Best value: 0.717333:  99%|█████████▉| 99/100 [06:06<00:05,  5.26s/it]

[I 2025-11-16 13:04:21,495] Trial 98 finished with value: 0.7140000000000001 and parameters: {'n_estimators': 717, 'max_depth': 5, 'learning_rate': 0.007267231922003335, 'min_child_weight': 3, 'subsample': 0.860227812387218, 'colsample_bytree': 0.7257860983889316, 'gamma': 0.18162382264924828, 'reg_alpha': 0.7039892764667254, 'reg_lambda': 0.8618905951430216}. Best is trial 51 with value: 0.7173333333333333.


Best trial: 51. Best value: 0.717333: 100%|██████████| 100/100 [06:10<00:00,  3.71s/it]


[I 2025-11-16 13:04:25,703] Trial 99 finished with value: 0.7066666666666667 and parameters: {'n_estimators': 676, 'max_depth': 4, 'learning_rate': 0.002946106630437834, 'min_child_weight': 3, 'subsample': 0.8756277477817229, 'colsample_bytree': 0.7719106008897197, 'gamma': 0.15442928290778057, 'reg_alpha': 0.7479546421571341, 'reg_lambda': 0.822483080271052}. Best is trial 51 with value: 0.7173333333333333.

Best XGBoost accuracy: 0.7173
Best parameters: {'n_estimators': 516, 'max_depth': 4, 'learning_rate': 0.011966949223279103, 'min_child_weight': 3, 'subsample': 0.8391969393161539, 'colsample_bytree': 0.8394626350861055, 'gamma': 0.14762912463169817, 'reg_alpha': 0.6726765817268627, 'reg_lambda': 0.5758401882979282}

XGBoost CV Accuracy: 0.7167 ± 0.0264
XGBoost F1-Macro: 0.5344
CPU times: total: 4min 16s
Wall time: 6min 25s


### 4.2 LightGBM with Hyperparameter Tuning

In [8]:
%%time

print("\n" + "="*80)
print("Training LightGBM with Hyperparameter Tuning")
print("="*80)

if OPTUNA_AVAILABLE:
    def objective_lgb(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'num_leaves': trial.suggest_int('num_leaves', 15, 255),
            'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 50),
            'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0),
            'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
            'lambda_l1': trial.suggest_float('lambda_l1', 0, 10),
            'lambda_l2': trial.suggest_float('lambda_l2', 0, 10),
            'random_state': RANDOM_STATE,
            'device': 'gpu',
            'verbose': -1
        }
        
        model = lgb.LGBMClassifier(**params)
        scores = cross_val_score(model, X, y_encoded, cv=skf, scoring='accuracy', n_jobs=-1)
        return scores.mean()
    
    study_lgb = optuna.create_study(direction='maximize', study_name='lightgbm')
    study_lgb.optimize(objective_lgb, n_trials=100, show_progress_bar=True)
    
    print(f"\nBest LightGBM accuracy: {study_lgb.best_value:.4f}")
    print(f"Best parameters: {study_lgb.best_params}")
    
    lgb_model = lgb.LGBMClassifier(**study_lgb.best_params)
    
else:
    param_dist = {
        'n_estimators': [100, 300, 500, 700, 1000],
        'num_leaves': [15, 31, 63, 127, 255],
        'learning_rate': [0.001, 0.01, 0.05, 0.1],
        'min_data_in_leaf': [10, 20, 30, 50],
        'feature_fraction': [0.6, 0.7, 0.8, 0.9, 1.0],
        'bagging_fraction': [0.6, 0.7, 0.8, 0.9, 1.0],
        'lambda_l1': [0, 0.1, 1, 10],
        'lambda_l2': [0, 0.1, 1, 10]
    }
    lgb_base = lgb.LGBMClassifier(random_state=RANDOM_STATE, device='gpu', verbose=-1)
    lgb_base = lgb.LGBMClassifier(random_state=RANDOM_STATE, verbose=-1)
    random_search = RandomizedSearchCV(lgb_base, param_dist, n_iter=100, cv=5,
                                      scoring='accuracy', n_jobs=-1, random_state=RANDOM_STATE)
    random_search.fit(X, y_encoded)
    
    print(f"\nBest LightGBM accuracy: {random_search.best_score_:.4f}")
    print(f"Best parameters: {random_search.best_params_}")
    
    lgb_model = random_search.best_estimator_

cv_results = cross_validate(lgb_model, X, y_encoded, cv=skf,
                           scoring=['accuracy', 'f1_macro'], return_train_score=True)

model_results['LightGBM'] = {
    'cv_accuracy': cv_results['test_accuracy'].mean(),
    'cv_std': cv_results['test_accuracy'].std(),
    'f1_macro': cv_results['test_f1_macro'].mean(),
    'model': lgb_model
}

print(f"\nLightGBM CV Accuracy: {model_results['LightGBM']['cv_accuracy']:.4f} ± {model_results['LightGBM']['cv_std']:.4f}")
print(f"LightGBM F1-Macro: {model_results['LightGBM']['f1_macro']:.4f}")

[I 2025-11-16 13:59:14,616] A new study created in memory with name: lightgbm



Training LightGBM with Hyperparameter Tuning


  0%|          | 0/100 [00:00<?, ?it/s]

Best trial: 0. Best value: 0.672667:   1%|          | 1/100 [00:54<1:29:24, 54.19s/it]

[I 2025-11-16 14:00:08,816] Trial 0 finished with value: 0.6726666666666666 and parameters: {'n_estimators': 498, 'num_leaves': 166, 'learning_rate': 0.0011151356450256468, 'min_data_in_leaf': 45, 'feature_fraction': 0.9504970109595525, 'bagging_fraction': 0.992082348903307, 'bagging_freq': 5, 'lambda_l1': 9.42243043792762, 'lambda_l2': 9.46289472157861}. Best is trial 0 with value: 0.6726666666666666.


Best trial: 0. Best value: 0.672667:   1%|          | 1/100 [03:41<6:04:49, 221.11s/it]

[W 2025-11-16 14:02:55,722] Trial 1 failed with parameters: {'n_estimators': 838, 'num_leaves': 53, 'learning_rate': 0.004747461821408483, 'min_data_in_leaf': 19, 'feature_fraction': 0.7169034614622614, 'bagging_fraction': 0.9452237521134776, 'bagging_freq': 3, 'lambda_l1': 3.059142688617631, 'lambda_l2': 2.6874401767788045} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\HP\anaconda3\envs\Vision\Lib\site-packages\optuna\study\_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "<timed exec>", line 23, in objective_lgb
  File "c:\Users\HP\anaconda3\envs\Vision\Lib\site-packages\sklearn\utils\_param_validation.py", line 218, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\HP\anaconda3\envs\Vision\Lib\site-packages\sklearn\model_selection\_validation.py", line 677, in cross_val_score
    cv_results = cross_validate(
         

KeyboardInterrupt: 

### 4.3 CatBoost with Hyperparameter Tuning

In [ ]:
%%time

print("\n" + "="*80)
print("Training CatBoost with Hyperparameter Tuning")
print("="*80)

if OPTUNA_AVAILABLE:
    def objective_cat(trial):
        params = {
            'iterations': trial.suggest_int('iterations', 500, 3000),
            'depth': trial.suggest_int('depth', 4, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
            'l2_leaf_reg': trial.suggest_int('l2_leaf_reg', 1, 9),
            'border_count': trial.suggest_int('border_count', 32, 254),
            'random_state': RANDOM_STATE,
            'task_type': 'GPU',
            'verbose': 0
        }
        
        model = CatBoostClassifier(**params)
        scores = cross_val_score(model, X, y_encoded, cv=skf, scoring='accuracy', n_jobs=-1)
        return scores.mean()
    
    study_cat = optuna.create_study(direction='maximize', study_name='catboost')
    study_cat.optimize(objective_cat, n_trials=50, show_progress_bar=True)  # Fewer trials due to longer training
    
    print(f"\nBest CatBoost accuracy: {study_cat.best_value:.4f}")
    print(f"Best parameters: {study_cat.best_params}")
    
    cat_model = CatBoostClassifier(**study_cat.best_params)
    
else:
    param_dist = {
        'iterations': [500, 1000, 2000, 3000],
        'depth': [4, 6, 8, 10],
        'learning_rate': [0.001, 0.01, 0.05, 0.1],
        'l2_leaf_reg': [1, 3, 5, 7, 9],
        'border_count': [32, 64, 128, 254]
    }
    cat_base = CatBoostClassifier(random_state=RANDOM_STATE, task_type='GPU', verbose=0)
    cat_base = CatBoostClassifier(random_state=RANDOM_STATE, verbose=0)
    random_search = RandomizedSearchCV(cat_base, param_dist, n_iter=50, cv=5,
                                      scoring='accuracy', n_jobs=-1, random_state=RANDOM_STATE)
    random_search.fit(X, y_encoded)
    
    print(f"\nBest CatBoost accuracy: {random_search.best_score_:.4f}")
    print(f"Best parameters: {random_search.best_params_}")
    
    cat_model = random_search.best_estimator_

cv_results = cross_validate(cat_model, X, y_encoded, cv=skf,
                           scoring=['accuracy', 'f1_macro'], return_train_score=True)

model_results['CatBoost'] = {
    'cv_accuracy': cv_results['test_accuracy'].mean(),
    'cv_std': cv_results['test_accuracy'].std(),
    'f1_macro': cv_results['test_f1_macro'].mean(),
    'model': cat_model
}

print(f"\nCatBoost CV Accuracy: {model_results['CatBoost']['cv_accuracy']:.4f} ± {model_results['CatBoost']['cv_std']:.4f}")
print(f"CatBoost F1-Macro: {model_results['CatBoost']['f1_macro']:.4f}")

### 4.4 Random Forest (Deep)

In [ ]:
%%time

print("\n" + "="*80)
print("Training Random Forest with Hyperparameter Tuning")
print("="*80)

param_grid = {
    'n_estimators': [500, 1000, 1500],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

rf_base = RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1)
random_search = RandomizedSearchCV(rf_base, param_grid, n_iter=50, cv=5,
                                  scoring='accuracy', n_jobs=-1, random_state=RANDOM_STATE)
random_search.fit(X, y_encoded)

print(f"\nBest Random Forest accuracy: {random_search.best_score_:.4f}")
print(f"Best parameters: {random_search.best_params_}")

rf_model = random_search.best_estimator_

cv_results = cross_validate(rf_model, X, y_encoded, cv=skf,
                           scoring=['accuracy', 'f1_macro'], return_train_score=True)

model_results['RandomForest'] = {
    'cv_accuracy': cv_results['test_accuracy'].mean(),
    'cv_std': cv_results['test_accuracy'].std(),
    'f1_macro': cv_results['test_f1_macro'].mean(),
    'model': rf_model
}

print(f"\nRandom Forest CV Accuracy: {model_results['RandomForest']['cv_accuracy']:.4f} ± {model_results['RandomForest']['cv_std']:.4f}")
print(f"Random Forest F1-Macro: {model_results['RandomForest']['f1_macro']:.4f}")

### 4.5 Extra Trees Classifier

In [ ]:
%%time

print("\n" + "="*80)
print("Training Extra Trees with Hyperparameter Tuning")
print("="*80)

param_grid = {
    'n_estimators': [500, 1000, 1500],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

et_base = ExtraTreesClassifier(random_state=RANDOM_STATE, n_jobs=-1)
random_search = RandomizedSearchCV(et_base, param_grid, n_iter=50, cv=5,
                                  scoring='accuracy', n_jobs=-1, random_state=RANDOM_STATE)
random_search.fit(X, y_encoded)

print(f"\nBest Extra Trees accuracy: {random_search.best_score_:.4f}")
print(f"Best parameters: {random_search.best_params_}")

et_model = random_search.best_estimator_

cv_results = cross_validate(et_model, X, y_encoded, cv=skf,
                           scoring=['accuracy', 'f1_macro'], return_train_score=True)

model_results['ExtraTrees'] = {
    'cv_accuracy': cv_results['test_accuracy'].mean(),
    'cv_std': cv_results['test_accuracy'].std(),
    'f1_macro': cv_results['test_f1_macro'].mean(),
    'model': et_model
}

print(f"\nExtra Trees CV Accuracy: {model_results['ExtraTrees']['cv_accuracy']:.4f} ± {model_results['ExtraTrees']['cv_std']:.4f}")
print(f"Extra Trees F1-Macro: {model_results['ExtraTrees']['f1_macro']:.4f}")

### 4.6 Neural Network (Deep)

In [7]:
%%time

print("\n" + "="*80)
print("Training Neural Network with Hyperparameter Tuning")
print("="*80)

param_grid = {
    'hidden_layer_sizes': [(256, 128, 64, 32), (512, 256, 128), (1024, 512, 256)],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate_init': [0.0001, 0.001, 0.01],
    'batch_size': [32, 64, 128]
}

nn_base = MLPClassifier(max_iter=1000, early_stopping=True, random_state=RANDOM_STATE)
random_search = RandomizedSearchCV(nn_base, param_grid, n_iter=30, cv=5,
                                  scoring='accuracy', n_jobs=-1, random_state=RANDOM_STATE)
random_search.fit(X, y_encoded)

print(f"\nBest Neural Network accuracy: {random_search.best_score_:.4f}")
print(f"Best parameters: {random_search.best_params_}")

nn_model = random_search.best_estimator_

cv_results = cross_validate(nn_model, X, y_encoded, cv=skf,
                           scoring=['accuracy', 'f1_macro'], return_train_score=True)

model_results['NeuralNetwork'] = {
    'cv_accuracy': cv_results['test_accuracy'].mean(),
    'cv_std': cv_results['test_accuracy'].std(),
    'f1_macro': cv_results['test_f1_macro'].mean(),
    'model': nn_model
}

print(f"\nNeural Network CV Accuracy: {model_results['NeuralNetwork']['cv_accuracy']:.4f} ± {model_results['NeuralNetwork']['cv_std']:.4f}")
print(f"Neural Network F1-Macro: {model_results['NeuralNetwork']['f1_macro']:.4f}")


Training Neural Network with Hyperparameter Tuning
CPU times: total: 2.17 s
Wall time: 3min 3s


KeyboardInterrupt: 

## 5. Base Model Comparison

In [ ]:
# Create comparison dataframe
comparison_df = pd.DataFrame([
    {
        'Model': name,
        'CV Accuracy': f"{results['cv_accuracy']:.4f}",
        'CV Std': f"{results['cv_std']:.4f}",
        'F1-Macro': f"{results['f1_macro']:.4f}"
    }
    for name, results in model_results.items()
]).sort_values('CV Accuracy', ascending=False)

print("\n" + "="*80)
print("BASE MODEL PERFORMANCE COMPARISON")
print("="*80)
print(comparison_df.to_string(index=False))

# Visualize results
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy comparison
models = list(model_results.keys())
accuracies = [model_results[m]['cv_accuracy'] for m in models]
stds = [model_results[m]['cv_std'] for m in models]

axes[0].barh(models, accuracies, xerr=stds, capsize=5)
axes[0].set_xlabel('CV Accuracy')
axes[0].set_title('Base Model Accuracy Comparison')
axes[0].set_xlim([min(accuracies) - 0.05, max(accuracies) + 0.05])

# F1-Score comparison
f1_scores = [model_results[m]['f1_macro'] for m in models]
axes[1].barh(models, f1_scores)
axes[1].set_xlabel('F1-Macro Score')
axes[1].set_title('Base Model F1-Score Comparison')
axes[1].set_xlim([min(f1_scores) - 0.05, max(f1_scores) + 0.05])

plt.tight_layout()
plt.savefig('visualizations/base_model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nVisualization saved to: visualizations/base_model_comparison.png")

## 6. Layer 2: Stacking Ensemble

In [ ]:
%%time

print("\n" + "="*80)
print("Building Stacked Ensemble")
print("="*80)

# Select top 5 base models
sorted_models = sorted(model_results.items(), key=lambda x: x[1]['cv_accuracy'], reverse=True)
top_5_models = sorted_models[:5]

print("\nTop 5 base models for stacking:")
for i, (name, results) in enumerate(top_5_models, 1):
    print(f"{i}. {name}: {results['cv_accuracy']:.4f}")

# Create estimators list for stacking
estimators = [(name, results['model']) for name, results in top_5_models]

# Meta-model 1: Logistic Regression
print("\n" + "-"*80)
print("Stacking with Logistic Regression Meta-Model")
print("-"*80)

stack_lr = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(max_iter=1000, random_state=RANDOM_STATE),
    cv=5,
    n_jobs=-1
)

cv_results_lr = cross_validate(stack_lr, X, y_encoded, cv=skf,
                              scoring=['accuracy', 'f1_macro'], return_train_score=True)

model_results['Stack_LogReg'] = {
    'cv_accuracy': cv_results_lr['test_accuracy'].mean(),
    'cv_std': cv_results_lr['test_accuracy'].std(),
    'f1_macro': cv_results_lr['test_f1_macro'].mean(),
    'model': stack_lr
}

print(f"Stacking (LogReg) CV Accuracy: {model_results['Stack_LogReg']['cv_accuracy']:.4f} ± {model_results['Stack_LogReg']['cv_std']:.4f}")
print(f"Stacking (LogReg) F1-Macro: {model_results['Stack_LogReg']['f1_macro']:.4f}")

# Meta-model 2: XGBoost
print("\n" + "-"*80)
print("Stacking with XGBoost Meta-Model")
print("-"*80)

stack_xgb = StackingClassifier(
    estimators=estimators,
    final_estimator=xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=RANDOM_STATE),
    cv=5,
    n_jobs=-1
)

cv_results_xgb = cross_validate(stack_xgb, X, y_encoded, cv=skf,
                               scoring=['accuracy', 'f1_macro'], return_train_score=True)

model_results['Stack_XGB'] = {
    'cv_accuracy': cv_results_xgb['test_accuracy'].mean(),
    'cv_std': cv_results_xgb['test_accuracy'].std(),
    'f1_macro': cv_results_xgb['test_f1_macro'].mean(),
    'model': stack_xgb
}

print(f"Stacking (XGB) CV Accuracy: {model_results['Stack_XGB']['cv_accuracy']:.4f} ± {model_results['Stack_XGB']['cv_std']:.4f}")
print(f"Stacking (XGB) F1-Macro: {model_results['Stack_XGB']['f1_macro']:.4f}")

## 7. Voting Ensemble

In [ ]:
%%time

print("\n" + "="*80)
print("Building Voting Ensemble")
print("="*80)

# Soft voting with top 5 models
voting_clf = VotingClassifier(
    estimators=estimators,
    voting='soft',
    n_jobs=-1
)

cv_results_vote = cross_validate(voting_clf, X, y_encoded, cv=skf,
                                scoring=['accuracy', 'f1_macro'], return_train_score=True)

model_results['Voting_Soft'] = {
    'cv_accuracy': cv_results_vote['test_accuracy'].mean(),
    'cv_std': cv_results_vote['test_accuracy'].std(),
    'f1_macro': cv_results_vote['test_f1_macro'].mean(),
    'model': voting_clf
}

print(f"Voting (Soft) CV Accuracy: {model_results['Voting_Soft']['cv_accuracy']:.4f} ± {model_results['Voting_Soft']['cv_std']:.4f}")
print(f"Voting (Soft) F1-Macro: {model_results['Voting_Soft']['f1_macro']:.4f}")

## 8. Final Model Selection and Comparison

In [ ]:
# Final comparison of all models
final_comparison_df = pd.DataFrame([
    {
        'Model': name,
        'CV Accuracy': results['cv_accuracy'],
        'CV Std': results['cv_std'],
        'F1-Macro': results['f1_macro']
    }
    for name, results in model_results.items()
]).sort_values('CV Accuracy', ascending=False)

print("\n" + "="*80)
print("FINAL MODEL PERFORMANCE COMPARISON (ALL MODELS)")
print("="*80)
print(final_comparison_df.to_string(index=False))

# Select best model
best_model_name = final_comparison_df.iloc[0]['Model']
best_model = model_results[best_model_name]['model']
best_accuracy = final_comparison_df.iloc[0]['CV Accuracy']

print(f"\n{'='*80}")
print(f"BEST MODEL: {best_model_name}")
print(f"Best CV Accuracy: {best_accuracy:.4f}")
print(f"{'='*80}")

# Visualize final comparison
plt.figure(figsize=(12, 8))
plt.barh(final_comparison_df['Model'], final_comparison_df['CV Accuracy'], 
         xerr=final_comparison_df['CV Std'], capsize=5)
plt.xlabel('CV Accuracy', fontsize=12)
plt.ylabel('Model', fontsize=12)
plt.title('Final Model Performance Comparison', fontsize=14, fontweight='bold')
plt.axvline(x=best_accuracy, color='r', linestyle='--', alpha=0.7, label=f'Best: {best_accuracy:.4f}')
plt.legend()
plt.tight_layout()
plt.savefig('visualizations/final_model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nVisualization saved to: visualizations/final_model_comparison.png")

## 9. Train Final Model on Full Dataset

In [ ]:
%%time

print("\n" + "="*80)
print(f"Training {best_model_name} on Full Dataset")
print("="*80)

# Train on full training data
best_model.fit(X, y_encoded)

# Training predictions
train_pred = best_model.predict(X)
train_accuracy = accuracy_score(y_encoded, train_pred)

print(f"\nTraining Accuracy: {train_accuracy:.4f}")
print(f"\nClassification Report (Training):")
print(classification_report(y_encoded, train_pred, target_names=le.classes_))

# Confusion matrix
cm = confusion_matrix(y_encoded, train_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=le.classes_, yticklabels=le.classes_)
plt.title(f'{best_model_name} - Confusion Matrix (Training)', fontsize=14, fontweight='bold')
plt.ylabel('Actual', fontsize=12)
plt.xlabel('Predicted', fontsize=12)
plt.tight_layout()
plt.savefig('visualizations/confusion_matrix_training.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nConfusion matrix saved to: visualizations/confusion_matrix_training.png")

## 10. Generate Test Predictions

In [ ]:
%%time

print("\n" + "="*80)
print("Generating Test Predictions")
print("="*80)

# Make predictions on test set
test_pred_encoded = best_model.predict(X_test)
test_pred = le.inverse_transform(test_pred_encoded)

# Get prediction probabilities
test_pred_proba = best_model.predict_proba(X_test)

print(f"\nTest predictions generated: {len(test_pred)}")
print(f"\nPrediction distribution:")
print(pd.Series(test_pred).value_counts())

# Create submission file
submission = pd.DataFrame({
    'id': test_ids,
    'drug_category': test_pred
})

submission_filename = f"submission_{best_model_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
submission.to_csv(submission_filename, index=False)

print(f"\nSubmission file created: {submission_filename}")
print(f"\nFirst 10 predictions:")
print(submission.head(10))

## 11. Save Models

In [ ]:
# Create models directory
os.makedirs('models', exist_ok=True)

# Save best model
best_model_filename = f"models/best_model_{best_model_name}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pkl"
joblib.dump(best_model, best_model_filename)
print(f"Best model saved: {best_model_filename}")

# Save label encoder
le_filename = "models/label_encoder.pkl"
joblib.dump(le, le_filename)
print(f"Label encoder saved: {le_filename}")

# Save all model results
results_filename = "models/model_results.pkl"
joblib.dump(model_results, results_filename)
print(f"All model results saved: {results_filename}")

# Save comparison dataframe
final_comparison_df.to_csv('models/model_comparison.csv', index=False)
print("Model comparison saved: models/model_comparison.csv")

## 12. Summary and Recommendations

In [ ]:
print("\n" + "="*80)
print("TRAINING SUMMARY")
print("="*80)

print(f"\nTotal models trained: {len(model_results)}")
print(f"\nBest performing model: {best_model_name}")
print(f"Best CV Accuracy: {best_accuracy:.4f}")
print(f"Best F1-Macro: {final_comparison_df.iloc[0]['F1-Macro']:.4f}")

print(f"\n{'='*80}")
print("TOP 5 MODELS")
print("="*80)
print(final_comparison_df.head().to_string(index=False))

print(f"\n{'='*80}")
print("RECOMMENDATIONS")
print("="*80)
print("\n1. Use the best model for final submission")
print("2. Consider ensemble of top 3-5 models for potentially better results")
print("3. Monitor for overfitting - check train vs validation gap")
print("4. If time permits, run additional Bayesian optimization trials")
print("5. Consider feature engineering if accuracy needs improvement")

print(f"\n{'='*80}")
print("FILES GENERATED")
print("="*80)
print(f"1. Submission: {submission_filename}")
print(f"2. Best model: {best_model_filename}")
print(f"3. Label encoder: {le_filename}")
print(f"4. Model results: {results_filename}")
print("5. Visualizations: visualizations/")
print("6. Model comparison: models/model_comparison.csv")

print(f"\n{'='*80}")
print("TRAINING COMPLETE!")
print("="*80)